<a href="https://colab.research.google.com/github/Elvenisboy/fuling_colaboratory/blob/master/resnet%E7%8C%AB%E7%8B%97%E5%A4%A7%E6%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!cp /content/drive/colab-drive/cats-vs-dogs/train.zip ./
!unzip ./train.zip

In [0]:
import keras
from keras.layers import Dense, Conv2D, MaxPooling2D,AveragePooling2D, Flatten, Dropout,BatchNormalization,Activation
from keras.models import Model, Input
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import os
from math import ceil

from keras.datasets import cifar10

In [0]:
def Unit(x, filters, pool=False):
  res = x
  if pool:
    x = MaxPooling2D(pool_size=(2,2))(x)
    res = Conv2D(filters=filters, kernel_size=[1,1], strides=(2,2), padding="same")(res)
  out = BatchNormalization()(x)
  out = Activation('relu')(out)
  out = Conv2D(filters=filters, kernel_size=[3,3], strides=(1,1), padding="same")(out)
  
  out = BatchNormalization()(out)
  out = Activation("relu")(out)
  out = Conv2D(filters=filters, kernel_size=[3,3], strides=(1,1), padding="same")(out)
  
  out = keras.layers.add([res, out])
  
  return out

In [0]:
def MiniModel(input_shape):
  images = Input(input_shape)
  net = Conv2D(filters=32, kernel_size=[3,3], strides=[1,1], padding="same")(images)
  net = Unit(net, 32)
  net = Unit(net, 32)  
  net = Unit(net, 32)
  
  net = Unit(net, 64, pool=True)
  net = Unit(net, 64)
  net = Unit(net, 64)  
  
  net = Unit(net, 128, pool=True)
  net = Unit(net, 128)  
  net = Unit(net, 128)
  
  net = Unit(net, 256, pool=True)
  net = Unit(net, 256)
  net = Unit(net, 256)
  
  net = BatchNormalization()(net)
  net = Activation("relu")(net)
  net = Dropout(0.25)(net)
  
  net = AveragePooling2D(pool_size=(4,4))(net)
  net = Flatten()(net)
  net = Dense(units=2, activation="softmax")(net)
  
  model = Model(inputs=images, outputs=net)
  
  return model


In [0]:
from keras.preprocessing import image
import numpy as np

def load_data(resize=32):
  images = os.listdir('./train/')
  num = len(images)
  train_data = []
  train_label = []
  test_data = []
  test_label = []

  for i in range(5000):
    if i % 2:
      img = image.load_img('./train/' + 'dog.' + str(i) + '.jpg', target_size=(resize, resize))
      x = image.img_to_array(img)
#       train_data[i] = x
      train_data.append(x)
      train_label.append(1)
    else:
      img = image.load_img('./train/' + 'cat.' + str(i) + '.jpg', target_size=(resize, resize))
      x = image.img_to_array(img)
      train_data.append(x)
      train_label.append(0)
  for i in range(5000, 10000):
    if i % 2:
      img = image.load_img('./train/' + 'dog.' + str(i) + '.jpg', target_size=(resize, resize))
      x = image.img_to_array(img)
      test_data.append(x)
      test_label.append(1)
    else:
      img = image.load_img('./train/' + 'cat.' + str(i) + '.jpg', target_size=(resize, resize))
      x = image.img_to_array(img)
      test_data.append(x)
      test_label.append(0)
  return train_data, train_label, test_data, test_label 

In [0]:

train_data, train_label, test_data, test_label = load_data(resize=32)
train_data, test_data = np.array(train_data), np.array(test_data)

In [0]:
train_label = keras.utils.to_categorical(train_label, 2)
test_label = keras.utils.to_categorical(test_label, 2)

In [30]:
input_shape = (32, 32, 3)
model = MiniModel(input_shape)

# Print a Summary of the model

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_44 (Conv2D)              (None, 32, 32, 32)   896         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_39 (BatchNo (None, 32, 32, 32)   128         conv2d_44[0][0]                  
__________________________________________________________________________________________________
activation_39 (Activation)      (None, 32, 32, 32)   0           batch_normalization_39[0][0]     
__________________________________________________________________________________________________
conv2d_45 

In [0]:
model.compile(optimizer=Adam(0.001), loss="categorical_crossentropy", metrics=["accuracy"])

epochs = 50

In [0]:
model.fit(x=train_data,
          y=train_label,
          batch_size=128,
         epochs=epochs)

Epoch 1/50
5000/5000 [==============================] - 315s 63ms/step - loss: 0.6453 - acc: 0.6294
Epoch 2/50
5000/5000 [==============================] - 309s 62ms/step - loss: 0.5741 - acc: 0.6996
Epoch 3/50
5000/5000 [==============================] - 315s 63ms/step - loss: 0.5093 - acc: 0.7462
Epoch 4/50
5000/5000 [==============================] - 318s 64ms/step - loss: 0.4140 - acc: 0.8084
Epoch 5/50
5000/5000 [==============================] - 317s 63ms/step - loss: 0.3093 - acc: 0.8624
Epoch 6/50
5000/5000 [==============================] - 307s 61ms/step - loss: 0.2347 - acc: 0.9062
Epoch 7/50
5000/5000 [==============================] - 314s 63ms/step - loss: 0.2981 - acc: 0.8718
Epoch 8/50
5000/5000 [==============================] - 312s 62ms/step - loss: 0.1296 - acc: 0.9522
Epoch 9/50
5000/5000 [==============================] - 319s 64ms/step - loss: 0.1147 - acc: 0.9558
Epoch 10/50
5000/5000 [==============================] - 310s 62ms/step - loss: 0.0562 - acc: 0.9820